In [1]:
import numpy as np

In [9]:
class Sigmoid:
    def __init__(self):
        # 学習すべきパラメータ
        self.params = []
        self.grads = []
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [10]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = x @ W + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = dout @ W.T
        dW = self.x.T @ dout
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [4]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        # np.random.randn(N, M) -> N x M行列を平均0,分散1の要素で初期化
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)

        # レイヤを生成
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]

        self.params = []
        for layer in self.layers:
            # ["a"] + ["b"] = ["a", "b"]
            self.params += layer.params

    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [5]:
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)
s

array([[ 0.23619298,  0.00229068,  0.65030179],
       [-0.05293865, -0.02328527,  0.57617352],
       [ 0.48715121, -0.24834236,  1.53056043],
       [ 0.04047487, -0.17391956,  1.07177243],
       [ 0.13119861, -0.06401978,  0.80785507],
       [ 0.52827798, -0.13322359,  1.14920659],
       [ 0.17397363,  0.14425726,  0.19802903],
       [-0.12506433,  0.17143604, -0.0159623 ],
       [ 0.1312029 ,  0.44498955, -0.4201796 ],
       [ 0.41918093,  0.26032664,  0.10244893]])

In [6]:
# この層自体に入力は複数ある
# 入力の次元と出力の次元は同じ
# 入力一つ一つに対しての確率を出力する
def softmax(x):
    # 入力が行列のとき
    if x.ndim == 2:
        # axisで軸の方向を決める
        # 0の場合は行ごと,1の場合は列ごとの最大値を求める
        # keepdimsをTrueにすることで二次配列のままになる
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x = x / x.sum(axis=1, keepdims=True)
    # 入力がベクトルのとき
    elif x.ndim == 1:
        x = x - np.max(x) # ここはオーバーフロー対策のためであって,理論的にはこの行は必要ない.ただしこの行があっても確率が変わらない?
        x = np.exp(x) / np.sum(np.exp(x))
    return x

In [7]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        # 二次配列に変更
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    # 教師データがone-hot-vectorの場合sizeが同じになる
    # 正解ラベルのインデックスのベクトルとなる
    if t.size == y.size:
        t = t.argmax(axis=1)
    # 行数を取得
    batch_size = y.shape[0]

    # np.arange(stop) 0 <= n < stop で間隔1でndarrayを生成
    # + 1e-7 これにより,予測データが0のときにエラーになるのを防ぐ
    # y[np.arange(batch_size), t] yが二次配列で
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [8]:
a = np.random.randn(1, 4)
a = a.reshape(1, a.size)
a

array([[ 0.56090121,  1.57959197, -0.19413277,  0.44233324]])

In [ ]:
# !! チェインルール
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grabs = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = x @ W
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = dout @ W.T
        dW = np.dot(self.x.T, dout)
        # 最初の[0]はもともとリストのなかでnp.zeros_likeをしているため [...]でdeepcopyしている
        self.grads[0][...] = dW
        return dx